In [ ]:
import csv
import os
import random
import pickle
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import sklearn
import warnings
import sys
from skbold.preproc import ConfoundRegressor
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet
import xgboost as xgb
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet

# dwMRI

In [ ]:

algorithms = ['svr', 'eNet', 'xgb', 'rf']
folds = ["0", "1", "2", "3", "4"]

if len(sys.argv) > 1:
    fold = int(sys.argv[1]) % 5
    modal = int(sys.argv[1]) // 5

algorithm_performance = {}
    
print(f'Reading files for fold {folds[fold]}')
    
features_train = pd.read_csv(f'/Stacking/features_train_level1_stacked/dti_all/features_train_level1_g_matched_fold_{folds[fold]}.csv')
target_real_train = pd.read_csv(f'/Stacking/g_factor/g_train_with_id_fold_{folds[fold]}.csv').merge(features_train, on='eid')['g'].values.reshape(-1, 1)
target_real_train_id = pd.read_csv(f'/Stacking/g_factor/g_train_with_id_fold_{folds[fold]}.csv').merge(features_train, on='eid')['eid']
features_train = features_train.drop(columns='eid')
    
features_test = pd.read_csv(f'/Stacking/features_test_level1_stacked/dti_all/features_test_level1_g_matched_fold_{folds[fold]}.csv')
target_real_test = pd.read_csv(f'/Stacking/g_factor/g_test_with_id_fold_{folds[fold]}.csv').merge(features_test, on='eid')['g'].values.reshape(-1, 1)
target_real_test_id = pd.read_csv(f'/Stacking/g_factor/g_test_with_id_fold_{folds[fold]}.csv').merge(features_test, on='eid')['eid']
features_test = features_test.drop(columns='eid')

print(f'Started fold {folds[fold]}')

scaler_features = StandardScaler()
scaler_target = StandardScaler()
     
print('Scaling')

target_real_train = scaler_target.fit_transform(target_real_train)
target_real_test = scaler_target.transform(target_real_test)
     
features_train = scaler_features.fit_transform(features_train)
features_test = scaler_features.transform(features_test)

with open(f'/Stacking/suppl/DTI_All_scaler_features_fold_{folds[fold]}.pkl', "wb") as f:
    pickle.dump(scaler_features, f)
    
with open(f'/Stacking/suppl/DTI_All_scaler_target_fold_{folds[fold]}.pkl', "wb") as f:
    pickle.dump(scaler_target, f)
    
for algorithm in algorithms:
    
    if algorithm =='svr':
        print('Started SVR')
        param = {'kernel' : ['rbf'],
                 'gamma' : ['scale','auto', 1e-08, 1e-07, 1e-06, 1e-05, 1e-04, 1e-03, 3e-08, 3e-07, 3e-06, 3e-05, 3e-04, 3e-03,6e-08, 6e-07, 6e-06, 6e-05, 6e-04, 6e-03],
                 'C': [1,6,9,10,12,15,20,50]}
        regressor = SVR()
            
    if algorithm == 'eNet':
        print('Started eNet')
        param = {'alpha': np.logspace(-6, 4, 500), 
                    'l1_ratio':np.linspace(0,1,100), 
                    'max_iter': [1000]}
        regressor = ElasticNet()
            
    if algorithm == 'xgb':
        print('Started XGb')
        param ={'booster': ['gbtree'],
                        'eta':[0.1,0.2,0.3],
                        'max_depth':[1,2,3,4,5,6],
                        'subsample':[0.3,0.6,1],
                        'lambda':[0,0.5,1],
                        'alpha':[0,0.5,1],
                        }
        regressor = xgb.XGBRegressor()
            
    if algorithm =='rf':
        print('Started RF')
        param ={'n_estimators': [5000],
                'max_depth':[1,2,3,4,5,6],
                'max_features':['auto','sqrt','log2']}
        regressor = RandomForestRegressor()
            
    model = GridSearchCV(regressor, param, cv=5, n_jobs = 17, verbose = 4)
    model.fit(features_train,target_real_train.ravel())

    with open(f'/Stacking/models/dti_all_new/DTI_All_stacked_{algorithm}_model_fold_{folds[fold]}.pkl', "wb") as f:
        pickle.dump(model, f)

    target_pred_second_level_train = model.predict(features_train)
    target_pred_second_level_test = model.predict(features_test)

    print(f"Fold = {folds[fold]}")
    print("----------")
    print("MSE = ", mean_squared_error(np.squeeze(target_real_test), np.squeeze(target_pred_second_level_test)))
    print("MAE = ", mean_absolute_error(np.squeeze(target_real_test), np.squeeze(target_pred_second_level_test)))
    print("R2 = ", r2_score(np.squeeze(target_real_test), np.squeeze(target_pred_second_level_test)))
    print("Pearson's r = ", pearsonr(np.squeeze(target_real_test), np.squeeze(target_pred_second_level_test)))
    print("----------")


    pd.DataFrame(target_pred_second_level_train, columns = ['g_pred_dti_all_stack_train'], index=target_real_train_id).reset_index().to_csv(f'/Stacking/g/dti_all_new/DTI_All_target_pred_2nd_level_{algorithm}_train_fold_{folds[fold]}.csv', index=False)
    pd.DataFrame(target_pred_second_level_test, columns = ['g_pred_dti_all_stack_test'], index=target_real_test_id).reset_index().to_csv(f'/Stacking/g/dti_all_new/DTI_All_target_pred_2nd_level_{algorithm}_test_fold_{folds[fold]}.csv', index=False)


    print(f'Model parameters for fold {folds[fold]}:', model.cv_results_['params'])
    print(f'Mean test score for fold {folds[fold]}:', model.cv_results_['mean_test_score'])
    print(f'Rank test score for fold {folds[fold]}:', model.cv_results_['rank_test_score'])
        
    print(model)
    print(f'Saving results for fold {folds[fold]}')
            
      
    print(f'Collecting results for fold {folds[fold]}')
        
    algorithm_performance['Algorithm'] = algorithm
    algorithm_performance['Fold'] = fold
    algorithm_performance['Best_params'] = model.best_params_
    algorithm_performance['MSE'] = mean_squared_error(np.squeeze(target_real_test), np.squeeze(target_pred_second_level_test))
    algorithm_performance['MAE'] = mean_absolute_error(np.squeeze(target_real_test), np.squeeze(target_pred_second_level_test))
    algorithm_performance['R2'] = r2_score(np.squeeze(target_real_test), np.squeeze(target_pred_second_level_test))
    algorithm_performance['Pearson r'] = pearsonr(np.squeeze(target_real_test), np.squeeze(target_pred_second_level_test))
    
        
    with open(f'/Stacking/result/dti_all_new/DTI_All_{algorithm}_stacked_result_fold_{folds[fold]}.csv', 'a', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=algorithm_performance.keys())
        writer.writerow(algorithm_performance)
            
    algorithm_performance.clear()

# rsMRI

In [ ]:
algorithms = ['svr', 'eNet', 'xgb', 'rf']
folds = ["0", "1", "2", "3", "4"]

if len(sys.argv) > 1:
    fold = int(sys.argv[1]) % 5
    algorithm = int(sys.argv[1]) // 5

algorithm_performance = {}
    
print(f'Reading files for fold {folds[fold]}')
    
features_train = pd.read_csv(f'/Stacking/features_train_level1_stacked/rs_idp_ts_best_metrics/features_train_level1_g_matched_fold_{folds[fold]}.csv')
target_real_train = pd.read_csv(f'/Stacking/g_factor/g_train_with_id_fold_{folds[fold]}.csv').merge(features_train, on='eid')['g'].values.reshape(-1, 1)
target_real_train_id = pd.read_csv(f'/Stacking/g_factor/g_train_with_id_fold_{folds[fold]}.csv').merge(features_train, on='eid')['eid']
print(target_real_train_id.shape)
features_train = features_train.drop(columns='eid')
print(features_train.shape)

    
features_test = pd.read_csv(f'/Stacking/features_test_level1_stacked/rs_idp_ts_best_metrics/features_test_level1_g_matched_fold_{folds[fold]}.csv')
target_real_test = pd.read_csv(f'/Stacking/g_factor/g_test_with_id_fold_{folds[fold]}.csv').merge(features_test, on='eid')['g'].values.reshape(-1, 1)
target_real_test_id = pd.read_csv(f'/Stacking/g_factor/g_test_with_id_fold_{folds[fold]}.csv').merge(features_test, on='eid')['eid']
print(target_real_test_id.shape)
features_test = features_test.drop(columns='eid')
print(features_test.shape)

print(f'Started fold {folds[fold]}')

scaler_features = StandardScaler()
scaler_target = StandardScaler()
     
print('Scaling')

target_real_train = scaler_target.fit_transform(target_real_train)
target_real_test = scaler_target.transform(target_real_test)
     
features_train = scaler_features.fit_transform(features_train)
features_test = scaler_features.transform(features_test)

with open(f'/Stacking/suppl/RS_IDP_Timeseries_best_metrics_scaler_features_fold_{folds[fold]}.pkl', "wb") as f:
    pickle.dump(scaler_features, f)
    
with open(f'/Stacking/suppl/RS_IDP_Timeseries_best_metrics_scaler_target_fold_{folds[fold]}.pkl', "wb") as f:
    pickle.dump(scaler_target, f)
    
if algorithms[algorithm] == 'svr':
    print('Started SVR')
    param = {'kernel' : ['rbf'],
                 'gamma' : ['scale','auto', 1e-08, 1e-07, 1e-06, 1e-05, 1e-04, 1e-03, 3e-08, 3e-07, 3e-06, 3e-05, 3e-04, 3e-03,6e-08, 6e-07, 6e-06, 6e-05, 6e-04, 6e-03],
                 'C': [1,6,9,10,12,15,20,50]}
    regressor = SVR()
            
elif algorithms[algorithm] == 'eNet':
    print('Started eNet')
    param = {'alpha': np.logspace(-6, 4, 500), 
                    'l1_ratio':np.linspace(0,1,100), 
                    'max_iter': [1000]}
    regressor = ElasticNet()
            
elif algorithms[algorithm] == 'xgb':
    print('Started XGb')
    param ={'booster': ['gbtree'],
                        'eta':[0.1,0.2,0.3],
                        'max_depth':[1,2,3,4,5,6],
                        'subsample':[0.3,0.6,1],
                        'lambda':[0,0.5,1],
                        'alpha':[0,0.5,1],
                        }
    regressor = xgb.XGBRegressor()
            
elif algorithms[algorithm] =='rf':
    print('Started RF')
    param ={'n_estimators': [5000],
                'max_depth':[1,2,3,4,5,6],
                'max_features':['auto','sqrt','log2']}
    regressor = RandomForestRegressor()
            
model = GridSearchCV(regressor, param, cv=5, verbose = 4) 
model.fit(features_train,target_real_train.ravel())

with open(f'/Stacking/models/RS_IDP_Timeseries_best_metrics_stacked_{algorithms[algorithm]}_model_fold_{folds[fold]}.pkl', "wb") as f:
    pickle.dump(model, f)

target_pred_second_level_train = model.predict(features_train)
target_pred_second_level_test = model.predict(features_test)

print(f"Fold = {folds[fold]}")
print("----------")
print("MSE = ", mean_squared_error(np.squeeze(target_real_test), np.squeeze(target_pred_second_level_test)))
print("MAE = ", mean_absolute_error(np.squeeze(target_real_test), np.squeeze(target_pred_second_level_test)))
print("R2 = ", r2_score(np.squeeze(target_real_test), np.squeeze(target_pred_second_level_test)))
print("Pearson's r = ", pearsonr(np.squeeze(target_real_test), np.squeeze(target_pred_second_level_test)))
print("----------")


pd.DataFrame(target_pred_second_level_train, columns = ['g_pred_rs_idp_ts_best_stack_train'], index=target_real_train_id).reset_index().to_csv(f'/Stacking/g/RS_IDP_Timeseries_best_metrics_target_pred_2nd_level_{algorithms[algorithm]}_train_fold_{folds[fold]}.csv', index=False)
pd.DataFrame(target_pred_second_level_test, columns = ['g_pred_rs_idp_ts_best_stack_test'], index=target_real_test_id).reset_index().to_csv(f'/Stacking/g/RS_IDP_Timeseries_best_metrics_target_pred_2nd_level_{algorithms[algorithm]}_test_fold_{folds[fold]}.csv', index=False)


print(f'Model parameters for fold {folds[fold]}:', model.cv_results_['params'])
print(f'Mean test score for fold {folds[fold]}:', model.cv_results_['mean_test_score'])
print(f'Rank test score for fold {folds[fold]}:', model.cv_results_['rank_test_score'])
        
print(model)
print(f'Saving results for fold {folds[fold]}')
                       
print(f'Collecting results for fold {folds[fold]}')
        
algorithm_performance['Algorithm'] = algorithms[algorithm]
algorithm_performance['Fold'] = fold
algorithm_performance['Best_params'] = model.best_params_
algorithm_performance['MSE'] = mean_squared_error(np.squeeze(target_real_test), np.squeeze(target_pred_second_level_test))
algorithm_performance['MAE'] = mean_absolute_error(np.squeeze(target_real_test), np.squeeze(target_pred_second_level_test))
algorithm_performance['R2'] = r2_score(np.squeeze(target_real_test), np.squeeze(target_pred_second_level_test))
algorithm_performance['Pearson r'] = pearsonr(np.squeeze(target_real_test), np.squeeze(target_pred_second_level_test))
    
        
with open(f'/Stacking/result/RS_IDP_Timeseries_best_metrics_{algorithms[algorithm]}_stacked_result_fold_{folds[fold]}.csv', 'a', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=algorithm_performance.keys())
    writer.writerow(algorithm_performance)    

# sMRI

In [ ]:
algorithms = ['svr', 'eNet', 'xgb', 'rf']
folds = ["0", "1", "2", "3", "4"]

if len(sys.argv) > 1:
    fold = int(sys.argv[1]) % 5
    algorithm = int(sys.argv[1]) // 5

algorithm_performance = {}
    
print(f'Reading files for fold {folds[fold]}')
    
features_train = pd.read_csv(f'/Stacking/features_train_level1_stacked/t1_t2_struct/features_train_level1_g_matched_fold_{folds[fold]}.csv')
target_real_train = pd.read_csv(f'/Stacking/g_factor/g_train_with_id_fold_{folds[fold]}.csv').merge(features_train, on='eid')['g'].values.reshape(-1, 1)
target_real_train_id = pd.read_csv(f'/Stacking/g_factor/g_train_with_id_fold_{folds[fold]}.csv').merge(features_train, on='eid')['eid']
print(target_real_train_id.shape)
features_train = features_train.drop(columns='eid')
print(features_train.shape)

    
features_test = pd.read_csv(f'/Stacking/features_test_level1_stacked/t1_t2_struct/features_test_level1_g_matched_fold_{folds[fold]}.csv')
target_real_test = pd.read_csv(f'/Stacking/g_factor/g_test_with_id_fold_{folds[fold]}.csv').merge(features_test, on='eid')['g'].values.reshape(-1, 1)
target_real_test_id = pd.read_csv(f'/Stacking/g_factor/g_test_with_id_fold_{folds[fold]}.csv').merge(features_test, on='eid')['eid']
print(target_real_test_id.shape)
features_test = features_test.drop(columns='eid')
print(features_test.shape)

print(f'Started fold {folds[fold]}')

scaler_features = StandardScaler()
scaler_target = StandardScaler()
     
print('Scaling')

target_real_train = scaler_target.fit_transform(target_real_train)
target_real_test = scaler_target.transform(target_real_test)
     
features_train = scaler_features.fit_transform(features_train)
features_test = scaler_features.transform(features_test)

with open(f'/Stacking/suppl/T1_T2_whole_brain_scaler_features_fold_{folds[fold]}.pkl', "wb") as f:
    pickle.dump(scaler_features, f)
    
with open(f'/Stacking/suppl/T1_T2_whole_brain_scaler_target_fold_{folds[fold]}.pkl', "wb") as f:
    pickle.dump(scaler_target, f)
    
if algorithms[algorithm] == 'svr':
    print('Started SVR')
    param = {'kernel' : ['rbf'],
                 'gamma' : ['scale','auto', 1e-08, 1e-07, 1e-06, 1e-05, 1e-04, 1e-03, 3e-08, 3e-07, 3e-06, 3e-05, 3e-04, 3e-03,6e-08, 6e-07, 6e-06, 6e-05, 6e-04, 6e-03],
                 'C': [1,6,9,10,12,15,20,50]}
    regressor = SVR()
            
elif algorithms[algorithm] == 'eNet':
    print('Started eNet')
    param = {'alpha': np.logspace(-6, 4, 500), 
                    'l1_ratio':np.linspace(0,1,100), 
                    'max_iter': [1000]}
    regressor = ElasticNet()
            
elif algorithms[algorithm] == 'xgb':
    print('Started XGb')
    param ={'booster': ['gbtree'],
                        'eta':[0.1,0.2,0.3],
                        'max_depth':[1,2,3,4,5,6],
                        'subsample':[0.3,0.6,1],
                        'lambda':[0,0.5,1],
                        'alpha':[0,0.5,1],
                        }
    regressor = xgb.XGBRegressor()
            
elif algorithms[algorithm] =='rf':
    print('Started RF')
    param ={'n_estimators': [5000],
                'max_depth':[1,2,3,4,5,6],
                'max_features':['sqrt','log2']} 
    regressor = RandomForestRegressor()
            
model = GridSearchCV(regressor, param, cv=5, verbose = 4) 
model.fit(features_train,target_real_train.ravel())

with open(f'/Stacking/models/T1_T2_whole_brain_stacked_{algorithms[algorithm]}_model_fold_{folds[fold]}.pkl', "wb") as f:
    pickle.dump(model, f)

target_pred_second_level_train = model.predict(features_train)
target_pred_second_level_test = model.predict(features_test)

print(f"Fold = {folds[fold]}")
print("----------")
print("MSE = ", mean_squared_error(np.squeeze(target_real_test), np.squeeze(target_pred_second_level_test)))
print("MAE = ", mean_absolute_error(np.squeeze(target_real_test), np.squeeze(target_pred_second_level_test)))
print("R2 = ", r2_score(np.squeeze(target_real_test), np.squeeze(target_pred_second_level_test)))
print("Pearson's r = ", pearsonr(np.squeeze(target_real_test), np.squeeze(target_pred_second_level_test)))
print("----------")


pd.DataFrame(target_pred_second_level_train, columns = ['g_pred_t1t2_stack_train'], index=target_real_train_id).reset_index().to_csv(f'/Stacking/g/T1_T2_whole_brain_target_pred_2nd_level_{algorithms[algorithm]}_train_fold_{folds[fold]}.csv', index=False)
pd.DataFrame(target_pred_second_level_test, columns = ['g_pred_t1t2_stack_test'], index=target_real_test_id).reset_index().to_csv(f'/Stacking/g/T1_T2_whole_brain_target_pred_2nd_level_{algorithms[algorithm]}_test_fold_{folds[fold]}.csv', index=False)


print(f'Model parameters for fold {folds[fold]}:', model.cv_results_['params'])
print(f'Mean test score for fold {folds[fold]}:', model.cv_results_['mean_test_score'])
print(f'Rank test score for fold {folds[fold]}:', model.cv_results_['rank_test_score'])
        
print(model)
print(f'Saving results for fold {folds[fold]}')
           
print(f'Collecting results for fold {folds[fold]}')
        
algorithm_performance['Algorithm'] = algorithms[algorithm]
algorithm_performance['Fold'] = fold
algorithm_performance['Best_params'] = model.best_params_
algorithm_performance['MSE'] = mean_squared_error(np.squeeze(target_real_test), np.squeeze(target_pred_second_level_test))
algorithm_performance['MAE'] = mean_absolute_error(np.squeeze(target_real_test), np.squeeze(target_pred_second_level_test))
algorithm_performance['R2'] = r2_score(np.squeeze(target_real_test), np.squeeze(target_pred_second_level_test))
algorithm_performance['Pearson r'] = pearsonr(np.squeeze(target_real_test), np.squeeze(target_pred_second_level_test))
    
        
with open(f'/Stacking/result/T1_T2_whole_brain_{algorithms[algorithm]}_stacked_result_fold_{folds[fold]}.csv', 'a', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=algorithm_performance.keys())
    writer.writerow(algorithm_performance)
            
algorithm_performance.clear()

# All Modalities

In [ ]:
algorithms = ['svr', 'eNet', 'xgb', 'rf']
folds = ["0", "1", "2", "3", "4"]

if len(sys.argv) > 1:
    fold = int(sys.argv[1]) % 5
    algorithm = int(sys.argv[1]) // 5

algorithm_performance = {}
    
print(f'Reading files for fold {folds[fold]}')
    
features_train = pd.read_csv(f'/Stacking/features_train_level1_stacked/all_modalities/features_train_level1_g_matched_fold_{folds[fold]}.csv')
target_real_train = pd.read_csv(f'/Stacking/g_factor/g_train_with_id_fold_{folds[fold]}.csv').merge(features_train, on='eid')['g'].values.reshape(-1, 1)
target_real_train_id = pd.read_csv(f'/Stacking/g_factor/g_train_with_id_fold_{folds[fold]}.csv').merge(features_train, on='eid')['eid']
print(target_real_train_id.shape)
features_train = features_train.drop(columns='eid')
print(features_train.shape)

    
features_test = pd.read_csv(f'/Stacking/features_test_level1_stacked/all_modalities/features_test_level1_g_matched_fold_{folds[fold]}.csv')
target_real_test = pd.read_csv(f'/Stacking/g_factor/g_test_with_id_fold_{folds[fold]}.csv').merge(features_test, on='eid')['g'].values.reshape(-1, 1)
target_real_test_id = pd.read_csv(f'/Stacking/g_factor/g_test_with_id_fold_{folds[fold]}.csv').merge(features_test, on='eid')['eid']
print(target_real_test_id.shape)
features_test = features_test.drop(columns='eid')
print(features_test.shape)

print(f'Started fold {folds[fold]}')

scaler_features = StandardScaler()
scaler_target = StandardScaler()
     
print('Scaling')

target_real_train = scaler_target.fit_transform(target_real_train)
target_real_test = scaler_target.transform(target_real_test)
     
features_train = scaler_features.fit_transform(features_train)
features_test = scaler_features.transform(features_test)

with open(f'/Stacking/suppl/All_modalities_scaler_features_fold_{folds[fold]}.pkl', "wb") as f:
    pickle.dump(scaler_features, f)
    
with open(f'/Stacking/suppl/All_modalities_scaler_target_fold_{folds[fold]}.pkl', "wb") as f:
    pickle.dump(scaler_target, f)
    
if algorithms[algorithm] == 'svr':
    print('Started SVR')
    param = {'kernel' : ['rbf'],
                 'gamma' : ['scale','auto', 1e-08, 1e-07, 1e-06, 1e-05, 1e-04, 1e-03, 3e-08, 3e-07, 3e-06, 3e-05, 3e-04, 3e-03,6e-08, 6e-07, 6e-06, 6e-05, 6e-04, 6e-03],
                 'C': [1,6,9,10,12,15,20,50]}
    regressor = SVR()
            
elif algorithms[algorithm] == 'eNet':
    print('Started eNet')
    param = {'alpha': np.logspace(-6, 4, 500), 
                    'l1_ratio':np.linspace(0,1,100), 
                    'max_iter': [1000]}
    regressor = ElasticNet()
            
elif algorithms[algorithm] == 'xgb':
    print('Started XGb')
    param ={'booster': ['gbtree'],
                        'eta':[0.1,0.2,0.3],
                        'max_depth':[1,2,3,4,5,6],
                        'subsample':[0.3,0.6,1],
                        'lambda':[0,0.5,1],
                        'alpha':[0,0.5,1],
                        }
    regressor = xgb.XGBRegressor()
            
elif algorithms[algorithm] =='rf':
    print('Started RF')
    param ={'n_estimators': [5000],
                'max_depth':[1,2,3,4,5,6],
                'max_features':['sqrt','log2']} 
    regressor = RandomForestRegressor()
            
model = GridSearchCV(regressor, param, cv=5, verbose = 4) 
model.fit(features_train,target_real_train.ravel())

with open(f'/Stacking/models/All_modalities_stacked_{algorithms[algorithm]}_model_fold_{folds[fold]}.pkl', "wb") as f:
    pickle.dump(model, f)

target_pred_second_level_train = model.predict(features_train)
target_pred_second_level_test = model.predict(features_test)

print(f"Fold = {folds[fold]}")
print("----------")
print("MSE = ", mean_squared_error(np.squeeze(target_real_test), np.squeeze(target_pred_second_level_test)))
print("MAE = ", mean_absolute_error(np.squeeze(target_real_test), np.squeeze(target_pred_second_level_test)))
print("R2 = ", r2_score(np.squeeze(target_real_test), np.squeeze(target_pred_second_level_test)))
print("Pearson's r = ", pearsonr(np.squeeze(target_real_test), np.squeeze(target_pred_second_level_test)))
print("----------")


pd.DataFrame(target_pred_second_level_train, columns = ['g_pred_mribest_stack_train'], index=target_real_train_id).reset_index().to_csv(f'/Stacking/g/All_modalities_target_pred_2nd_level_{algorithms[algorithm]}_train_fold_{folds[fold]}.csv', index=False)
pd.DataFrame(target_pred_second_level_test, columns = ['g_pred_mribest_stack_test'], index=target_real_test_id).reset_index().to_csv(f'/Stacking/g/All_modalities_target_pred_2nd_level_{algorithms[algorithm]}_test_fold_{folds[fold]}.csv', index=False)


print(f'Model parameters for fold {folds[fold]}:', model.cv_results_['params'])
print(f'Mean test score for fold {folds[fold]}:', model.cv_results_['mean_test_score'])
print(f'Rank test score for fold {folds[fold]}:', model.cv_results_['rank_test_score'])
        
print(model)
print(f'Saving results for fold {folds[fold]}')
                      
print(f'Collecting results for fold {folds[fold]}')
        
algorithm_performance['Algorithm'] = algorithms[algorithm]
algorithm_performance['Fold'] = fold
algorithm_performance['Best_params'] = model.best_params_
algorithm_performance['MSE'] = mean_squared_error(np.squeeze(target_real_test), np.squeeze(target_pred_second_level_test))
algorithm_performance['MAE'] = mean_absolute_error(np.squeeze(target_real_test), np.squeeze(target_pred_second_level_test))
algorithm_performance['R2'] = r2_score(np.squeeze(target_real_test), np.squeeze(target_pred_second_level_test))
algorithm_performance['Pearson r'] = pearsonr(np.squeeze(target_real_test), np.squeeze(target_pred_second_level_test))
    
        
with open(f'/Stacking/result/All_modalities_{algorithms[algorithm]}_stacked_result_fold_{folds[fold]}.csv', 'a', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=algorithm_performance.keys())
    writer.writerow(algorithm_performance)
            
algorithm_performance.clear()